In [2]:
using DataFrames, CSV, Dates, XLSX, RollingFunctions, GLM, LinearAlgebra, MLBase, Combinatorics

In [2]:
"""
    getRainData(OBXL::Int)
Returns a dataframe with the rain data from the specific hyetogram

# Arguments
- `OBXL::Int`: The number corresponding to the hyetogram of interest.
"""

function getRainData(OBXL::Int)
   return CSV.read("C:/Users/takum/Downloads/Recherche/TakumiTherville/Pluies/Hyétogramme_treated/OBXL"*string(OBXL)*"_treated",DataFrame)
end

getRainData (generic function with 1 method)

In [6]:
"""
    getCSOData(site::String)
Returns a dataframe with the CSO data from the specific site

# Arguments
- `site::String`: The name of the CSO site of interest.
"""

function getCSOData(site::String)
    data_cso = CSV.read("C:/Users/takum/Downloads/Recherche/AlexandrineLanson/Données/cso_raw.csv",DataFrame)
    selectSite!(data_cso, "Site", site)
    return data_cso
end

getCSOData (generic function with 1 method)

In [1]:
"""
    getCSOData(site::Vector{String})
Returns a dataframe with the CSO data from the specific sites

# Arguments
- `site::Vector{String}`: The names of the CSO sites of interest.
"""

function getCSOData(site::Vector{String})
    data_cso = CSV.read("C:/Users/takum/Downloads/Recherche/AlexandrineLanson/Données/cso_raw.csv",DataFrame)
    selectSite!(data_cso, "Site", site)
    return data_cso
end

getCSOData (generic function with 1 method)

In [7]:
"""
    getData(OBXL::Int, site::String)
Returns one dataframe with the Rain data and another with the CSO data.

# Arguments
- `OBXL::Int`: The number corresponding to the hyetogram of interest.
- `site::String`: The name of the CSO site of interest.
"""

function getData(OBXL::Int, site::String)
    return getRainData(OBXL), getCSOData(site) 
end

getData (generic function with 1 method)

In [8]:
"""
    removeCalibration!(df::DataFrame, colValidity::String, calibrationTag::String)
Removes all rows for which the hyetogram was in a calibration state.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colValidity::String`: The name of the column of `df` containing Validity data.
- `calibrationTag::String` : The tag corresponding to the calibration state.
"""

function removeCalibration!(df::DataFrame, colValidity::String, calibrationTag::String)
    filter!(colValidity => x -> ismissing(x) || !occursin(calibrationTag, x), df)
end

removeCalibration! (generic function with 1 method)

In [9]:
"""
    fillDates_RainData(df::DataFrame, colDate::String, colRain::String, from::DateTime, to::DateTime, by::Int = 5)
Fills the dataframe with the missing Dates in a given period.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colDate::String`: The name of the column of `df` containing the dates.
- `colRain::String`: The name of the column of `df` containing the rainfall data.
- `from::DateTime` : The beginning date of the period of interest to be filled in.
- `to::DateTime` : The end of the period of interest to be filled in.
- `by::Int` : Time discretisation step in minutes, 5 minutes per default.

#Implementation 
The function first creates all the missing dates in the period. 
Then sets the rainfall value for the new dates to 0.
"""

function fillDates_RainData(df::DataFrame, colDate::String, colRain::String, from::DateTime, to::DateTime; by::Int = 5)
    dates = DataFrame(colDate => collect(from:Dates.Minute(by):to))
    df_new = outerjoin(dates, df, on = Symbol(colDate))
    df_new[ismissing.(df_new[!,colRain]),colRain] .= 0
    
    return df_new
end

fillDates_RainData (generic function with 1 method)

In [10]:
"""
    fillDates_RainData(df::DataFrame, colDate::String, colRain::String, from::Int, to::Int; by::Int = 5)
Fills the dataframe with the missing Dates between two years.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colDate::String`: The name of the column of `df` containing the dates.
- `colRain::String`: The name of the column of `df` containing the rainfall data.
- `from::DateTime` : The starting year of the dataframe to be returned.
- `to::DateTime` : The ending year of the dataframe to be returned.
- `by::Int` : Time discretisation step in minutes, 5 minutes per default.

#Implementation 
The function first creates all the missing dates between the two years. 
Let us note that the end is not included in the period.
Then sets the rainfall value for the new dates to 0.
"""

function fillDates_RainData(df::DataFrame, colDate::String, colRain::String, from::Int, to::Int; by::Int = 5)
    dates = DataFrame(colDate => collect(DateTime(from,1,1,0,0,0):Dates.Minute(by):DateTime(to,1,1,0,0,0)))
    df_new = outerjoin(dates, df, on = Symbol(colDate))
    df_new[ismissing.(df_new[!,colRain]),colRain] .= 0
    
    return df_new
end

fillDates_RainData (generic function with 2 methods)

In [11]:
"""
    createAccumulationData(df::DataFrame,colDay::String, colRain::String, varNames::Vector{Symbol}, maxNb::Vector{Int})
Creates the maximum accumulations dataframe for the requested accumulation durations.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colDay::String`: The name of the column of `df` containing the days.
- `colRain::String`: The name of the column of `df` containing the rainfall data.
- `varNames::Vector{Symbol}` : The vector of names to be given to the new max accumulation columns.
- `maxNb::Vector{Int}` : The number of times the initial discretisation step can be taken for each accumulation duration.
"""

function createAccumulationData(df::DataFrame,colDay::String, colRain::String, varNames::Vector{Symbol}, maxNb::Vector{Int})
    
    
    
    df[!,varNames[1]] = df[!,colRain]

    for i in 2:length(varNames)
        df[!, varNames[i]] = running(sum, df[!,colRain], maxNb[i])
    end
    
    #Set the column order
    df = df[!, vcat(Symbol(colDay), varNames)]
    
    #Split the data by Date
    df_split = groupby(df, Symbol(colDay))

    #Create the empty dataframe with the daily rain accumulations and add a row for each day with the maximum accumulation
    df_daily = hcat(DataFrame(Symbol(colDay) => Date[]),DataFrame([name => Float64[] for name in vcat(varNames)]))

    
    for i in 1:length(df_split)
        this_day_maxs = describe(df_split[i]).max
        push!(df_daily, this_day_maxs)
    end
    
    #Rename the day column to Date to allow for easy joining later with the cso dataframe
    rename!(df_daily, Symbol(colDay) => :Date)
    
    return df_daily
    
end

createAccumulationData (generic function with 1 method)

In [12]:
"""
    selectSite!(df::DataFrame, colSite::String, site::String)
Selects a certain CSO site data.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colSite::String`: The name of the column of `df` containing which CSO Site each data entry is from.
- `site::String` : The site of interest.
"""

function selectSite!(df::DataFrame, colSite::String, site::String)
    filter!(colSite => x -> x == site, df)
end

selectSite! (generic function with 1 method)

In [ ]:
"""
    selectSite!(df::DataFrame, colSite::String, site::Vector{String})
Selects certain CSO sites data.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colSite::String`: The name of the column of `df` containing which CSO Site each data entry is from.
- `site::Vector{String}` : The sites of interest.
"""

function selectSite!(df::DataFrame, colSite::String, site::Vector{String})
    filter!(colSite => x -> x in site, df)
end

In [13]:
"""
    selectCause!(df::DataFrame, colCause::String, cause::String, acceptMissing::Bool)
Selects a CSO occurence cause in the data.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colCause::String`: The name of the column of `df` containing the cause of each CSO.
- `cause::String` : The cause of interest.
- `acceptMissing::Bool` : Whether or not Missing entries of the cause should be kept in the data.
"""


function selectCause!(df::DataFrame, colCause::String, cause::String, acceptMissing::Bool)
    if acceptMissing 
        filter!(colCause => x -> ismissing(x) || x == cause, df)
    else
        filter!(colCause => x ->  !ismissing(x) && x == cause, df)
    end
end

selectCause! (generic function with 1 method)

In [14]:
"""
    createDayCol!(df::DataFrame, colDay::String, colDateTime::String)
Creates a column corresponding to the day.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colDay::String`: The name to be given to the new day column.
- `colDateTime::String` : The name of the column of `df` containing the date and time.
"""

function createDayCol!(df::DataFrame, colDay::String, colDateTime::String)
    df[!,colDay] = [Date(Dates.year(i),Dates.month(i),Dates.day(i)) for i in df[!,colDateTime]]
end

createDayCol! (generic function with 1 method)

In [7]:
"""
    toMMRainfall!(df::DataFrame, colRain::String)
Converts the rainfall data from mm/hour to mm*10^-1. 

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colRain::String`: The name of the column of `df` containing the rainfall data.
"""

function toMMRainfall!(df::DataFrame, colRain::String)
   df[!,colRain] = df[!,colRain]*10/12 
end

toMMRainfall! (generic function with 1 method)

In [16]:
"""
    createOverflowCol!(df::DataFrame, colCSO::String, colDuration::String)
Creates a boolean column indicating whether or not a CSO occured from the CSO durations.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colCSO::String`: The name to be given to the new CSO column.
- `colDuration::String` : The name of the column of `df` containing the duration of the CSOs.
"""

function createOverflowCol!(df::DataFrame, colCSO::String, colDuration::String)
    df[!,colCSO] .= (df[!,colDuration] .!= 0)
end

createOverflowCol! (generic function with 1 method)

In [17]:
"""
    deleteCSOAnomalies!(df::DataFrame, col::String, colCSO::String, thresholdCol::Float64, CSOstate::bool)
Delete rows in the dataframe for which a certain entry is lower than a threshold in a specific CSO state.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `col::String`: The name of the column of `df` containing the data of interest.
- `colCSO::String` : The name of the column of `df` containing the CSO occurence.
- `thrsholdCol::Float64` : The threshold under (including) which  the values of the data of interest should be deleted.
- `CSOstate::bool` : The occurence (boolean) of CSO of interest.
"""

function deleteCSOAnomalies!(df::DataFrame, col::String, colCSO::String, thresholdCol::Float64, CSOstate::Bool)
    filter!(row-> (row[colCSO] != CSOstate) || (row[col] >= thresholdCol && row[colCSO] == CSOstate), df)
end

deleteCSOAnomalies! (generic function with 1 method)

In [9]:
"""
    preTreatRainData(df::DataFrame)
Returns the pretreated rainfall data.

# Arguments
- `df::DataFrame`: The dataframe containing the rainfall data.
"""

function preTreatRainData(df::DataFrame)
    
    removeCalibration!(df, "Validity", "CALIBRATION")
    
    df_full = fillDates_RainData(df, "Date", "Rainfall", 2013, 2022)

    toMMRainfall!(df_full, "Rainfall")

    select!(df_full, Not([:Validity]))
    
    createDayCol!(df_full, "Day", "Date")
    
    df_full = sort(df_full, :Date)
    
    varnames = [:d5min, :d10min, :d15min, :d20min, :d25min,
            :d30min, :d35min, :d40min, :d45min, :d50min,
            :d55min, :d1h, :d2h, :d3h, :d4h,
            :d6h, :d8h, :d12h, :d18h, :d24h]

    maxNb = [1, 2, 3, 4, 5,
        6, 7, 8, 9, 10,
        11, 12, 24, 36, 48,
        72, 96, 144, 216, 288]

    df_daily = createAccumulationData(df_full,"Day","Rainfall",varnames,maxNb)
    
    return df_daily
    
end

preTreatRainData (generic function with 1 method)

In [2]:
"""
    preTreatRainData(df::DataFrame, varnames::Vector{Symbol}, maxNb::Vector{Int})
Returns the pretreated rainfall data with specified accumulation variables.

# Arguments
- `df::DataFrame`: The dataframe containing the rainfall data.
- `varnames::Vector{Symbol}`: The names of the accumulation data to be computed.
- `maxNb::Vector{Int}`: The number of periods corresponding to the accumulation.
- `by::Int`: The time discretization step of the data in minutes.
"""

function preTreatRainData(df::DataFrame, varnames::Vector{Symbol}, maxNb::Vector{Int})
    
    removeCalibration!(df, "Validity", "CALIBRATION")
    
    df_full = df

    select!(df_full, Not([:Validity]))
    
    createDayCol!(df_full, "Day", "Date")
    
    df_full = sort(df_full, :Date)

    df_daily = createAccumulationData(df_full,"Day","Rainfall",varnames,maxNb)
    
    return df_daily
    
end

LoadError: UndefVarError: DataFrame not defined

In [19]:
"""
    preTreatFullData!(df::DataFrame)
Pretreats the full data resulting of the joined rainfall and CSO data.

# Arguments
- `df::DataFrame`: The dataframe containing the full data.
"""

function preTreatFullData!(df::DataFrame)
    select!(df, Not([:Monitored, :Comment, :Site, :Code]))

    dropmissing!(df, :Duration)
    dropmissing!(df, :d5min)

    createOverflowCol!(df, "CSO", "Duration")

    #deleteCSOAnomalies!(df, "d24h", "CSO", 5.0, true)
end

preTreatFullData! (generic function with 1 method)

In [3]:
"""
    preTreatFullData!(df::DataFrame, colDuration::String, cold5::String, cold24::String, colToDel::Vector{String})
Pretreats the full data resulting of the joined rainfall and CSO data.

# Arguments
- `df::DataFrame`: The dataframe containing the full data.
- `colDuration::String`: The name of the column of `df` containing the durations.
- `cold5::String`: The name of the column of `df` containing the minimal accumulation data.
- `cold24::String`: The name of the column of `df` containing the maximal accumulation data.
- `colToDel::Vector{String}`: The names of the column of `df` to be deleted.
"""

function preTreatFullData!(df::DataFrame, colDuration::String, cold5::String, cold24::String, colToDel::Vector{String})
    select!(df, Not(Symbol.(colToDel)))

    dropmissing!(df, Symbol(colDuration))
    dropmissing!(df, Symbol(cold5))

    createOverflowCol!(df, "CSO", colDuration)

    #deleteCSOAnomalies!(df, cold24, "CSO", 5.0, true)
end

LoadError: UndefVarError: DataFrame not defined

In [3]:
"""
    splitDataset(df::DataFrame, colDay::String ,year::Int)
Split the dataframe into a training set and a test set.

# Arguments
- `df::DataFrame`: The dataframe containing the data.
- `colDay::String`: The name of the column of `df` containing the dates.
- `year::Int` : The year at which the data should be split.

#Implementation
The data is split such that the test data starts with the specified year.
"""

function splitDataset(df::DataFrame, colDay::String ,year::Int)
    train = df[df[!,colDay] .< Date(year,1,1),:]
    test = df[df[!,colDay] .>= Date(year,1,1),:]
    
    return train, test
end

splitDataset (generic function with 1 method)

In [2]:
"""
    F1(predictions, trueValues)
Computes the F1 score given the predictions and the true values.

# Arguments
- `predictions`: The predicted valuesin a 1D table form.
- `trueValues`: The true values in a 1D table form.
"""

function F1(predictions, trueValues)
    
    tp = 0
    for i in 1:length(trueValues)
        if trueValues[i] == 1 && predictions[i] == 1
            tp = tp + 1
        end
    end
    
    return 2*tp/(2*tp + sum(predictions.!= trueValues))
    
end

F1 (generic function with 1 method)

In [29]:
"""
    compute_F1(M, test)
Computes the maximum F1 score using different thresholds. 

# Arguments
- `M`: The predicted model in a 1D table form.
- `test::DataFrame`: The true values in a 1D table form.
- `colToPredict::String` : The name of the column of `test` containing the variable of interest's data.
"""

function compute_F1(M, test::DataFrame, colToPredict::String)
    
    predictedvals = [predict(M,test)[i] .> j for i in 1:size(test)[1], j in 0.1:0.01:0.6]
    groundtruthvals = test[!,Symbol.(colToPredict)]
    
    F1_scores = [F1(predictedvals[:,j], groundtruthvals) for j in 1:size(predictedvals,2)]
    #print("(index, the corresponding best threshold)")
    show((argmax(F1_scores), collect(0.1:0.01:0.6)[argmax(F1_scores)]))
    
    return maximum(F1_scores), collect(0.1:0.01:0.6)[argmax(F1_scores)]
    
end

compute_F1 (generic function with 1 method)

In [2]:
"""
    compute_best_F1_threshold(M, test)
Computes the threshold leading to the maximum F1 score.

# Arguments
- `M`: The predicted model in a 1D table form.
- `test::DataFrame`: The true values in a 1D table form.
- `colToPredict::String` : The name of the column of `test` containing the variable of interest's data.
"""

function compute_best_F1_threshold(M, test::DataFrame, colToPredict::String)
    
    predictedvals = [predict(M,test)[i] .> j for i in 1:size(test)[1], j in 0.1:0.01:0.6]
    groundtruthvals = test[!,colToPredict]
    
    F1_scores = [F1(predictedvals[:,j], groundtruthvals) for j in 1:size(predictedvals,2)]
    #print("(index, the corresponding best threshold)")
    show((argmax(F1_scores), collect(0.1:0.01:0.6)[argmax(F1_scores)]))
    
    return collect(0.1:0.01:0.6)[argmax(F1_scores)]
    
end

LoadError: UndefVarError: DataFrame not defined

In [5]:
"""
    compute_MSE(M, test)
Computes the Mean Squared Error.

# Arguments
- `M`: The predicted model in a 1D table form.
- `test::DataFrame`: The true values in a 1D table form.
- `colToPredict::String` : The name of the column of `df` containing the variable of interest's data.
"""

function compute_MSE(M, test::DataFrame, colToPredict::String)
    
    predictedvals = predict(M,test)
    groundtruthvals = test[!,colToPredict]
    
    MSE = sum((predictedvals-groundtruthvals).^2)/(length(predictedvals))
    
    return MSE
    
end

LoadError: UndefVarError: DataFrame not defined

In [4]:
"""
    compute_RMSD(M, test)
Computes the Root mean squared deviation.

# Arguments
- `M`: The predicted model in a 1D table form.
- `test::DataFrame`: The true values in a 1D table form.
- `colToPredict::String` : The name of the column of `df` containing the variable of interest's data.
"""

function compute_RMSD(M, test::DataFrame, colToPredict::String)
    
    return sqrt(compute_MSE(M, test, colToPredict))
    
end

LoadError: UndefVarError: DataFrame not defined

In [5]:
"""
    getLogisticModel(train::DataFrame, toPredict::String, predictors::Vector{String})
Returns the logistic regression model with the given predictors and variable of interest.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
"""

function getLogisticModel(train::DataFrame, toPredict::String, predictors::Vector{String})
    
    
    formula = Term(Symbol(toPredict)) ~ sum(term.(predictors))
    local m
    try 
        m = glm(formula, train, Bernoulli(), LogitLink(), maxiter = 100)
    catch e
        return 0.0
    else
        return m
    end
    
end

LoadError: UndefVarError: DataFrame not defined

In [ ]:
"""
    getNormalModel(train::DataFrame, toPredict::String, predictors::Vector{String}, link::String)
Returns the normal regression model with the given predictors and variable of interest.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `link::String` : The link function to be used.
"""

function getNormalModel(train::DataFrame, toPredict::String, predictors::Vector{String}, link::String)
    
    formula = Term(Symbol(toPredict)) ~ sum(term.(predictors))
    if link == "inverse"
        m = glm(formula, train, Normal(), InverseLink(), maxiter = 100)
    elseif link == "log"
        m = glm(formula, train, Normal(), LogLink(), maxiter = 100)
    elseif link == "identity"
        m = glm(formula, train, Normal(), maxiter = 100)
    end
    
    
    return m
    
end

In [3]:
"""
    getGammaModel(train::DataFrame, toPredict::String, predictors::Vector{String})
Returns the gamma regression model with the given predictors and variable of interest.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
"""

function getGammaModel(train::DataFrame, toPredict::String, predictors::Vector{String},link::String)
    
    formula = Term(Symbol(toPredict)) ~ sum(term.(predictors))
    if link == "inverse"
        m = glm(formula, train, Gamma(), InverseLink(), maxiter = 1000)
    elseif link == "log"
        m = glm(formula, train, Gamma(), LogLink(), maxiter = 1000)
    end
    
    return m
    
end

LoadError: UndefVarError: DataFrame not defined

In [ ]:
"""
    getLinearModel(train::DataFrame, toPredict::String, predictors::Vector{String})
Returns the linear regression model with the given predictors and variable of interest.

# Arguments
- `train::DataFrame`: The dataframe containing the training data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
"""

function getLinearModel(train::DataFrame, toPredict::String, predictors::Vector{String})
    
    formula = Term(Symbol(toPredict)) ~ sum(term.(predictors))
    m = lm(formula, train)
    
    return m
    
end

In [8]:
"""
    getTreeModel(train::DataFrame, toPredict::String, predictors::Vector{String}, max_depth::Int)
Returns the regression tree model with the given predictors and variable of interest.

# Arguments
- `train::DataFrame`: The dataframe containing the training data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `max_depth::Int` : The maximum tree depth.
"""

function getTreeModel(train::DataFrame, toPredict::String, predictors::Vector{String}, max_depth::Int)
    
    modelDT = DecisionTreeClassifier(max_depth = max_depth)
    tree = DecisionTree.fit!(modelDT, Matrix(train[:, predictors]), train[:,toPredict])

    return tree
    
end

LoadError: UndefVarError: DataFrame not defined

In [20]:
"""
    extensiveSearchTree(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, max_depth::Int)
Returns the F1 score of all possible logistic regression models in decreasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.

#Implementation
The returned F1 score corresponds to the best possible F1 score with different prediction thresholds in the range of 0.1 
to 0.6 with a 0.01 increment. 
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchTree(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, max_depth::Int)
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
    
    l = 1
    for i in combinations

        model = getTreeModel(train, toPredict, i, max_depth)
        pred = DecisionTree.predict(model, Matrix(test[:, i]))
        
        
        s = rmsd(pred, test[:, toPredict])
        scores[l] = s
        l = l+1
        
    end
    
    
    
    return sort(DataFrame(Variables = collect(combinations), RMSD = scores), :RMSD)
end

LoadError: UndefVarError: DataFrame not defined

In [8]:
"""
    extensiveSearchLogistic(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String})
Returns the F1 score of all possible logistic regression models in decreasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `threshold::Float64` : The value of the classification threshold. A value of 0 indicates that the best classification threshold should be computed and used 

#Implementation
The returned F1 score corresponds to the best possible F1 score with different prediction thresholds in the range of 0.1 
to 0.6 with a 0.01 increment. 
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchLogistic(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, threshold::Float64)
    
    
    combinations = Combinatorics.combinations(varNames)
    f_scores = Vector{Float64}(undef, 2^length(varNames)-1)
    if threshold == 0
        thresholds = Vector{Float64}(undef, 2^length(varNames)-1)
    end
    
    l = 1
    for i in combinations
        
        model = getLogisticModel(train, toPredict, i)
        
        if typeof(model) == Float64
            println("Error in model fitting")
            
            f = -Inf 
            if threshold == 0.0
                thresholds[l] = 1.0
            end
        else
            if threshold != 0.0
                f = F1([predict(model,test)[i] .> threshold for i in 1:size(test)[1]], test[!,toPredict])
            else
            
                function fobj(θ::Real)
                    scores = convert(Vector{Float64}, MLBase.predict(model, train))
                    r = roc(train.CSO, scores, θ)
                    return -f1score(r)
                end
            
                res = optimize(fobj, 0.0, 1.0)
                thresholds[l] = res.minimizer
            
                f = - res.minimum
            
            end
        end
        

        f_scores[l] = f
        l = l+1
        #print(" The corresponding F score is ")
        #println(f)
    end
    
    if threshold == 0
        return sort(DataFrame(Variables = collect(combinations), F1 = f_scores, Threshold = thresholds), :F1, rev = true)
    else
        return sort(DataFrame(Variables = collect(combinations), F1 = f_scores), :F1, rev = true)
    end
end

LoadError: UndefVarError: DataFrame not defined

In [4]:
"""
    extensiveSearchLinear(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String)
Returns the mean squared error of all possible linear regression models in increasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `criteria::String` : The goodness of fit indicator to be used.

#Implementation
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchLinear(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String)
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
    
    if (criteria == "RMSD_CSO" )|| (criteria == "MSE_CSO")
        l = 1
        for i in combinations
            model = getLinearModel(train[train[!,toPredict] .> 0,:], toPredict, i)
            mse = compute_MSE(model, test[test[!,toPredict] .> 0, :], toPredict)
            scores[l] = mse
            l = l+1
            #print(" The corresponding MSE is ")
            #println(f)
        end
        
        if criteria == "MSE"
            return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
        elseif criteria == "RMSD"
            return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
        end
    end
        
    l = 1
    for i in combinations
        model = getLinearModel(train, toPredict, i)
        mse = compute_MSE(model, test, toPredict)
        scores[l] = mse
        l = l+1
        #print(" The corresponding MSE is ")
        #println(f)
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
        
end

LoadError: UndefVarError: DataFrame not defined

In [ ]:
"""
    extensiveSearchNormal(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String, link::String)
Returns the mean squared error of all possible gaussian regression models in increasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `criteria::String` : The goodness of fit indicator to be used.
- `link::String` : The link function to be used.

#Implementation
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchNormal(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String, link::String)
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
        
    l = 1
    for i in combinations
        model = getNormalModel(train, toPredict, i, link)

        mse = compute_MSE(model, test, toPredict)
        scores[l] = mse
        l = l+1
        #print(" The corresponding MSE is ")
        #println(f)
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
        
end

In [ ]:
"""
    extensiveSearchGamma(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String, link::String)
Returns the mean squared error or rmsd of all possible Gamma regression models in increasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `criteria::String` : The goodness of fit indicator to be used.
- `link::String` : The link function to be used.

#Implementation
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchGamma(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String, link::String)
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
        
    l = 1
    for i in combinations
        model = getGammaModel(train, toPredict, i, link)

        mse = compute_MSE(model, test, toPredict)
        scores[l] = mse
        l = l+1
        #print(" The corresponding MSE is ")
        #println(f)
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
        
end

In [11]:
"""
    getBestModelSiteOBXL(site::String, OBXL::Int, year::Int, Vars::Vector{String})
Returns the best models based on the F1 score.

# Arguments
- `site::String`: The name of the CSO site of interest
- `OBXL::Int`: The number corresponding to the hyetogram to be used.
- `year::Int` : The year when the test data starts.
- `Vars::Vector{String}`: The names of the variables to be used in the models.

#Implementation
The variables in Vars are supposed to be one of the next [:d5min, :d10min, :d15min, :d20min,
:d25min, :d30min, :d35min, :d40min, :d45min, :d50min, :d55min, :d1h, :d2h, :d3h, :d4h,
:d6h, :d8h, :d12h, :d18h, :d24h].
"""

function getBestModelSiteOBXL(site::String, OBXL::Int, year::Int, Vars::Vector{String})
    
    println("Importing data for OBXL"*string(OBXL)*" and site "*site)
    
    data_rain, data_cso = getData(OBXL,site)
    
    data_rain_daily = preTreatRainData(data_rain)

    selectCause!(data_cso, "Code", "P", true)
    
    data = outerjoin(data_rain_daily, data_cso, on = :Date) 
    
    preTreatFullData!(data)
    
    train, test = splitDataset(data, "Date", 2020)
    
    f_scores = extensiveSearchLogistic(train, test, "CSO", Vars)
    
    sort(f_scores, :F1, rev=true)
    
    return groupby(f_scores, :F1)[1]
    
end

getBestModelSiteOBXL (generic function with 1 method)

In [26]:
"""
    bestModelFormatting(sdf::SubDataFrame)
Formats the input SubDataFrame containing the best model information into a tuple with the score and the best models.

# Arguments
- `sdf::SubDataFrame`: the sub dataframe containing the best models with their score. 

"""

function bestModelFormatting(sdf::SubDataFrame)
    return (sdf.F1[1], sdf.Variables)
end

bestModelFormatting (generic function with 1 method)

In [27]:
"""
    getBestModelSitesOBXLs(site::String, OBXL::Int, year::Int, Vars::Vector{String})
Returns the best models for each Site and Hyetogram based on the F1 score.

# Arguments
- `sites::Vector{String}`: The names of the CSO sites of interest
- `OBXLs::Vector{Int}`: The numbers corresponding to the hyetograms to be used.
- `year::Int` : The year when the test data starts.
- `Vars::Vector{String}`: The names of the variables to be used in the models.

#Implementation
The variables in Vars are supposed to be one of the next [:d5min, :d10min, :d15min, :d20min,
:d25min, :d30min, :d35min, :d40min, :d45min, :d50min, :d55min, :d1h, :d2h, :d3h, :d4h,
:d6h, :d8h, :d12h, :d18h, :d24h].

"""

function getBestModelSitesOBXLs(sites::Vector{String}, OBXLs::Vector{Int}, year::Int, Vars::Vector{String})
    n = length(OBXLs)
    m = length(sites)
    
    out = [bestModelFormatting(getBestModelSiteOBXL(sites[i], OBXLs[j], year, Vars)) for i in 1:m, j in 1:n]
    out = hcat(DataFrame(Site = sites), DataFrame(out, string.(OBXLs)))
    
    return out
end

getBestModelSitesOBXLs (generic function with 1 method)

In [28]:
"""
    getBestModelSitesOBXLs2(site::String, OBXL::Int, year::Int, Vars::Vector{String})
Returns the best models for each Site and Hyetogram based on the F1 score.
# Arguments
- `sites::Vector{String}`: The names of the CSO sites of interest
- `OBXLs::Vector{Int}`: The numbers corresponding to the hyetograms to be used.
- `year::Int` : The year when the test data starts.
- `Vars::Vector{String}`: The names of the variables to be used in the models.

#Implementation
The variables in Vars are supposed to be one of the next [:d5min, :d10min, :d15min, :d20min,
:d25min, :d30min, :d35min, :d40min, :d45min, :d50min, :d55min, :d1h, :d2h, :d3h, :d4h,
:d6h, :d8h, :d12h, :d18h, :d24h].

The function getBestModelSiteOBXL(site::String, OBXL::Int, year::Int, Vars::Vector{String}) was not used as to minimize
the run time. Let us note that this function should be later modified as to modify the outer for loop depending on the 
size of its arguments. Furthermore, the function body should be simplified into a simple function call which would take 
either a single site/OBXl and a vector of OBXLs/sites and compute the desired best models and F1 scores.Those 
implementations will be made later on.

"""

function getBestModelSitesOBXLs2(sites::Vector{String}, OBXLs::Vector{Int}, year::Int, Vars::Vector{String})
    n = length(OBXLs)
    m = length(sites)
    out = Array{Any,2}(undef, m, n)
    
    for i in 1:m
        data_cso = getCSOData(sites[i])
        
        for j in 1:n
            println("Importing data for OBXL"*string(OBXLs[j])*" and site "*sites[i])
            
            data_rain = getRainData(OBXLs[j])
            
            data_rain_daily = preTreatRainData(data_rain)

            selectCause!(data_cso, "Code", "P", true)
    
            data = outerjoin(data_rain_daily, data_cso, on = :Date) 
        
            preTreatFullData!(data)
    
            train, test = splitDataset(data, "Date", 2020)
    
            f_scores = extensiveSearchLogistic(train, test, "CSO", Vars)
            
            out[i,j] = bestModelFormatting(groupby(f_scores, :F1)[1])
            
            print("Best Models for "*sites[i]*" OBXL"*string(OBXLs[j])*" ")
            println(out[i,j])
            
        end
        
        println("Best Models for "*sites[i])
        println(out[i,:])
        
        CSV.write(sites[i]*"_bestModels.csv", out[i,:])
    end
    
    out = hcat(DataFrame(Site = sites), DataFrame(out, string.(OBXLs)))
    
    return out
end

getBestModelSitesOBXLs2 (generic function with 1 method)

In [6]:
"""
    forwardStepSearch(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, strictly::Bool)
Returns the F1 score and the corresponding model of a forward model search.

# Arguments
- `train::DataFrame`: The dataframe containing the training data for the model.
- `test::DataFrame`: The dataframe containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `varNames::Vector{String}` : The names of the column of `df` containing the predictors.
- `strictly::Bool` : A boolean indicating whether the search should be strictly increasing or not.

#Implementation
The search uses the F1 score as a criteria. 
The returned F1 score corresponds to the best possible F1 score with different prediction thresholds in the range of 0.1 
to 0.6 with a 0.01 increment. 
The data is returned in the form of a 2 element tuple with first the F1 score and then the corresponding model.
"""

function forwardStepSearch(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, strictly::Bool)
    
    selectedVars = String[]
    unselectedVars = varNames
    
    bestF1 = 0
    
    improved = true
    
    if strictly
    
        while improved 
        
            thisBestF1 = bestF1
            improved = false
            thisBestPred = 0
        
            for newVar in unselectedVars  
            
                thisF1 = compute_F1(getLogisticModel(train, toPredict, vcat(selectedVars,newVar)), test, toPredict)
                println(newVar, thisF1)
            
                if thisF1 > thisBestF1
                    thisBestF1 = thisF1
                    thisBestPred = newVar
                    improved = true
                end
            end
        
        
            if improved 
                println("The variable "*thisBestPred*" has been added with F1 "*string(thisBestF1))
                selectedVars = vcat(selectedVars,thisBestPred)
                deleteat!(unselectedVars, findall(x->x== thisBestPred,unselectedVars))
                bestF1 = thisBestF1
            end
        
        
        end
        
    else
         selectedVarsString = String[]
        
         while improved 
        
            thisBestF1 = bestF1
            improved = false
            thisBestPred = 0
            equal = false
        
            for newVar in unselectedVars  
            
                thisF1 = compute_F1(getLogisticModel(train, toPredict, vcat(selectedVars,newVar)), test, toPredict)
                println(newVar, thisF1)
            
                if (thisF1 > thisBestF1)
                    thisBestF1 = thisF1
                    thisBestPred = newVar
                    improved = true
                    equal = false
                    
                elseif (thisF1 == thisBestF1) && (improved == false)
                    equal = true
                    thisBestF1 = thisF1
                    thisBestPred = newVar
                    improved = true
                end
            end
        
        
            if improved && equal
                println("The variable "*thisBestPred*" has been added with same F1 "*string(thisBestF1))
                selectedVars = vcat(selectedVars,thisBestPred)
                selectedVarsString = vcat(selectedVarsString,thisBestPred*"s")
                deleteat!(unselectedVars, findall(x->x== thisBestPred,unselectedVars))
                bestF1 = thisBestF1
            elseif improved 
                println("The variable "*thisBestPred*" has been added with F1 "*string(thisBestF1))
                selectedVars = vcat(selectedVars,thisBestPred)
                selectedVarsString = vcat(selectedVarsString,thisBestPred)
                deleteat!(unselectedVars, findall(x->x== thisBestPred,unselectedVars))
                bestF1 = thisBestF1
            end
        
        
        end
        
    end
    
    return (bestF1, selectedVarsString)
end

LoadError: UndefVarError: DataFrame not defined

In [14]:
"""
    forwardStepSearch(site::String, OBXL::Int, year::Int, Vars::Vector{String}, strictly::Bool)
Returns the F1 score and the corresponding model of a forward model search.
# Arguments
- `sites::Vector{String}`: The names of the CSO sites of interest
- `OBXLs::Vector{Int}`: The numbers corresponding to the hyetograms to be used.
- `year::Int` : The year when the test data starts.
- `Vars::Vector{String}`: The names of the variables to be used in the models.
- `strictly::Bool` : A boolean indicating whether the search should be strictly increasing or not.

#Implementation
The search uses the F1 score as a criteria. 
The returned F1 score corresponds to the best possible F1 score with different prediction thresholds in the range of 0.1 
to 0.6 with a 0.01 increment. 
The data is returned in the form of a 2 element tuple with first the F1 score and then the corresponding model.
"""


function forwardStepSearch(site::String, OBXL::Int, year::Int, Vars::Vector{String}, strictly::Bool)
    
    println("Importing data for OBXL"*string(OBXL)*" and site "*site)
    
    data_rain, data_cso = getData(OBXL,site)
    
    data_rain_daily = preTreatRainData(data_rain)

    selectCause!(data_cso, "Code", "P", true)
    
    data = outerjoin(data_rain_daily, data_cso, on = :Date) 
    
    preTreatFullData!(data)
    
    train, test = splitDataset(data, "Date", 2020)
    
    f_score = forwardStepSearch(train, test, "CSO", Vars, strictly)
    
    return f_score
end

forwardStepSearch (generic function with 3 methods)

In [1]:
using Combinatorics, Optim
"""
    extensiveSearchLogistic(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, threshold::Float64)
Returns the F1 score of all possible logistic regression models in decreasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `threshold::Float64` : The value of the classification threshold. A value of 0 indicates that the best classification threshold should be computed and used 

#Implementation
The returned F1 score corresponds to the F1 score of the logistic model using the best classification threshold on the training data. 
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchLogistic(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, thresholds::Vector{Float64})
    
    println(thresholds)
    
    combinations = Combinatorics.combinations(varNames)
    f_scores = Array{Float64}(undef, 2^length(varNames)-1, length(thresholds))
    
    l = 1
    for i in combinations
        
        println(l)
        
        model = getLogisticModel(train, toPredict, i)
        
        scores = convert(Vector{Float64}, MLBase.predict(model, test))

        f_scores[l,:] =  [f1score(roc(test[!,toPredict], scores, threshold)) for threshold in thresholds]
        l = l+1
        
    end
    
    
    d = DataFrame(hcat(collect(combinations),f_scores), :auto)
    rename!(d, vcat(:Variables, Symbol.(string.(thresholds))))
    
    return d
end







"""
count_flags(s::String, flag::String)

counts the number of flags `flag` in string `s`.
"""
function count_flags(s::String, flag::String)
counter = 0
for i in 1:length(s)
  if occursin(flag, s)
    s = replace(s, flag=> "", count=1)
    counter+=1
  else
    break
  end
end
return counter
end

LoadError: UndefVarError: DataFrame not defined

In [ ]:
using DataTables

"""
    extensiveSearchLogisticRUS(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, n::Int)
Returns the F1 score of all possible logistic regression models in decreasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `n::Int` : The number of Randomly Undersampled datasets to use

#Implementation
The mean F1 score is computed from the individual F1 score on the test data of the models trained on 
100 randomly undersampled datasets.
The data is returned in the form of a 2 column matrix with the best performing models on average on top.
"""

function extensiveSearchLogisticRUS(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, n::Int)
    
    
    combinations = Combinatorics.combinations(varNames)
    f_scores = Array{Float64}(undef, 2^length(varNames)-1, n)
    
    first = DataFrame(DataTable(undersample(row->row[Symbol(toPredict)], DataTable(train))))
    l_first = size(first)[1]
    rus = hcat(first, DataFrame(RUS = Int.(ones(l_first))))
    
    for i in 2:n
        rus = vcat(rus, hcat(DataFrame(DataTable(undersample(row->row[Symbol(toPredict)], DataTable(train)))), DataFrame(RUS = Int.(i .* ones(l_first)))))
    end
    
    rus_j = groupby(rus, :RUS)
    
    i = 1
    for k in combinations
        
        
        for j in rus_j
            
            local model
            
            
            model = getLogisticModel(DataFrame(j), toPredict, k)
            if model == 0.0
                println("Error in model fitting")
                f = 0
            else 
                f = f1score(roc(test[!,toPredict], convert(Vector{Float64}, MLBase.predict(model, test)), .5))
            end
            
            f_scores[i, j.RUS[1]] = f
            
        end
        
        println(k, mean(f_scores[i,:]))
        
        i = i+1
        
        
    end
    
    return sort(DataFrame(Variables = collect(combinations), F1 = [mean(f_scores[i,:]) for i in 1:(2^length(varNames)-1)]), :F1, rev = true)
    
end

In [ ]:
"""
    extensiveSearchLogisticROS(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, n::Int)
Returns the F1 score of all possible logistic regression models in decreasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `n::Int` : The number of Randomly Undersampled datasets to use

#Implementation
The mean F1 score is computed from the individual F1 score on the test data of the models trained on 
100 randomly undersampled datasets.
The data is returned in the form of a 2 column matrix with the best performing models on average on top.
"""

function extensiveSearchLogisticROS(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, n::Int)
    
    
    combinations = Combinatorics.combinations(varNames)
    f_scores = Array{Float64}(undef, 2^length(varNames)-1, n)
    
    first = DataFrame(DataTable(oversample(row->row[Symbol(toPredict)], DataTable(train))))
    l_first = size(first)[1]
    rus = hcat(first, DataFrame(RUS = Int.(ones(l_first))))
    
    for i in 2:n
        rus = vcat(rus, hcat(DataFrame(DataTable(oversample(row->row[Symbol(toPredict)], DataTable(train)))), DataFrame(RUS = Int.(i .* ones(l_first)))))
    end
    
    rus_j = groupby(rus, :RUS)
    
    i = 1
    for k in combinations
        
        
        for j in rus_j
            
            local model
            
             
            model = getLogisticModel(DataFrame(j), toPredict, k)
            if model == 0.0
                println("Error in model fitting")
                f = 0
            else 
                f = f1score(roc(test[!,toPredict], convert(Vector{Float64}, MLBase.predict(model, test)), .5))
            end
            
            f_scores[i, j.RUS[1]] = f
            
        end
        
        println(k, mean(f_scores[i,:]))
        
        i = i+1
        
        
    end
    
    return sort(DataFrame(Variables = collect(combinations), F1 = [mean(f_scores[i,:]) for i in 1:(2^length(varNames)-1)]), :F1, rev = true)
    
end

In [5]:
using PyCall

"""
    extensiveSearchLogisticSMOTE(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, n::Int)
Returns the F1 score of all possible logistic regression models in decreasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `n::Int` : The number of Randomly Undersampled datasets to use

#Implementation
The mean F1 score is computed from the individual F1 score on the test data of the models trained on 
100 randomly undersampled datasets.
The data is returned in the form of a 2 column matrix with the best performing models on average on top.
"""

function extensiveSearchLogisticSMOTE(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, n::Int)
    
    
    combinations = Combinatorics.combinations(varNames)
    f_scores = Array{Float64}(undef, 2^length(varNames)-1, n)
    
    imb = pyimport("imblearn") 
    smt = imb.over_sampling.SMOTE(sampling_strategy = 1.0)
    
    d_first, first_CSO = smt.fit_resample(Array(select(train, Not([:Date, :Site, :CSO]))), train.CSO) 
    col_names_full = names(select(train, Not([:Date, :Site, :CSO])))
    d_first = DataFrame(hcat(d_first, first_CSO), vcat(col_names_full,String.([:CSO])))
    
    l_first = size(d_first)[1]
    smote = hcat(d_first, DataFrame(SMOTE = Int.(ones(l_first))))
    
    for i in 2:n
        
        current, current_CSO = smt.fit_resample(Array(select(train, Not([:Date, :Site, :CSO]))), train.CSO)
        current = DataFrame(hcat(current, current_CSO), vcat(col_names_full,String.([:CSO])))
        smote = vcat(smote, hcat(current, DataFrame(SMOTE = Int.(i .* ones(l_first)))))
    end
    
    smote_j = groupby(smote, :SMOTE)
    
    i = 1
    for k in combinations
        
        
        for j in smote_j
            
            local model
            
             
            model = getLogisticModel(DataFrame(j), toPredict, k)
            if model == 0.0
                println("Error in model fitting")
                f = 0
            else 
                f = f1score(roc(test[!,toPredict], convert(Vector{Float64}, MLBase.predict(model, test)), .5))
            end
            
            f_scores[i, j.SMOTE[1]] = f
            
        end
        
        println(k, mean(f_scores[i,:]))
        
        i = i+1
        
        
    end
    
    return sort(DataFrame(Variables = collect(combinations), F1 = [mean(f_scores[i,:]) for i in 1:(2^length(varNames)-1)]), :F1, rev = true)
    
end

LoadError: UndefVarError: DataFrame not defined

In [ ]:
"""
    extensiveSearchLinear(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String)
Returns the mean squared error of all possible linear regression models in increasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `criteria::String` : The goodness of fit indicator to be used.

#Implementation
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchLinear(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String)
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
    
    if (criteria == "RMSD_CSO" )|| (criteria == "MSE_CSO")
        l = 1
        for i in combinations
            model = getLinearModel(train[train[!,toPredict] .> 0,:], toPredict, i)
            mse = compute_MSE(model, test[test[!,toPredict] .> 0, :], toPredict)
            scores[l] = mse
            l = l+1
        end
        
        if criteria == "MSE"
            return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
        elseif criteria == "RMSD"
            return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
        end
    end
        
    l = 1
    for i in combinations
        model = getLinearModel(train, toPredict, i)
        mse = compute_MSE(model, test, toPredict)
        scores[l] = mse
        
        println(i, scores[l])
        l = l+1
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
        
end

In [ ]:
"""
    extensiveSearchGamma(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String, link::String)
Returns the mean squared error or rmsd of all possible Gamma regression models in increasing order.

# Arguments
- `train::DataFrame`: The datafram containing the training data for the model.
- `test::DataFrame`: The datafram containing the test data for the model.
- `toPredict::String`:The name of the column of `df` containing the variable of interest to be predicted.
- `predictors::Vector{String}` : The names of the column of `df` containing the predictors.
- `criteria::String` : The goodness of fit indicator to be used.
- `link::String` : The link function to be used.

#Implementation
The data is returned in the form of a 2 column matrix with the best performing models on top.
"""

function extensiveSearchGamma(train::DataFrame, test::DataFrame, toPredict::String, varNames::Vector{String}, criteria::String, link::String)
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
        
    l = 1
    for i in combinations
        model = getGammaModel(train, toPredict, i, link)

        mse = compute_MSE(model, test, toPredict)
        scores[l] = mse
        
        println(i, mse)
        l = l+1
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
        
end

In [ ]:
using Distributions, SpecialFunctions

function log_likelihood_bc(x, lambda)
    n = length(x)
    if lambda == 0
        y = log.(x)
    else 
        y = (x .^lambda .-1)/lambda
    end
    
    sigma_sq = var(y, corrected = false)
    mu = mean(y)
    
    return -n/2*log(sigma_sq) - 1/(2*sigma_sq)*(y .-mu)'*(y .- mu) + (lambda-1)*sum(log.(x))
end

function log_likelihood_gamma_bc(x, lambda)
    n = length(x)
    if lambda == 0
        y = log.(x)
    else 
        y = (x .^lambda .-1)/lambda
    end
    
    sigma_sq = var(y, corrected = false)
    mu = mean(y)
    
    beta = mu/sigma_sq
    alpha = mu^2/sigma_sq
    
    return alpha*n*log(beta) -n*log(gamma(alpha)) + (alpha-1)*sum(log.(y)) - beta*sum(y) + (lambda-1)*sum(log.(x))
end


function getCI(log_likelihood, lambda_0, x)
    
    low = optimize(lambda -> abs(2*(log_likelihood(x, lambda_0) - log_likelihood(x, lambda))-cquantile(Chisq(1), .05)), lambda_0-5, lambda_0)
    upp = optimize(lambda -> abs(2*(log_likelihood(x, lambda_0) - log_likelihood(x, lambda))-cquantile(Chisq(1), .05)), lambda_0, lambda_0+5)
    
    return (low.minimizer, lambda_0, upp.minimizer)
    
end

In [ ]:
function transf_lin_mod(data,test_year,lambda, toPredict, varNames, criteria)
    
    train = filter(row -> year(row.Date) != test_year, data)
    test = filter(row -> year(row.Date) == test_year, data)
    
    
    if lambda != 0
        train[!, :new] = train.Duration .^(lambda)
    else 
        train[!, :new] = log.(train.Duration)
    end
    
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
    
    
    l = 1
    for i in combinations
         model = getLinearModel(train, "new", i)
            
        if lambda != 0
            preds = predict(model, test)
            MSE_test = sum( (real.((complex.(preds) ).^(1/lambda)) - test.Duration).^2 ) / (nrow(test) )
            #MSE_train = sum( (predict(model, train) .^(1/lambda) - train.new .^(1/lambda)).^2 ) / (nrow(train) -length(β̂))
            #MSE_full = sum( (predict(model, data) .^(1/lambda) - data.new .^(1/lambda)).^2 ) / (nrow(data) -length(β̂))
        else
            MSE_test = sum( (exp.(predict(model, test)) - test.Duration).^2 ) / (nrow(test) )
            #MSE_train = sum( (exp.(predict(model, train)) - exp.(train.new)).^2 ) / (nrow(train) -length(β̂))
            #MSE_full = sum( (exp.(predict(mdoel, data)) - exp.(data.new)).^2 ) / (nrow(data) -length(β̂))
        end
        mse = MSE_test
        
        scores[l] = real(mse)
        println(i, mse)
        l = l+1
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
    
end

In [ ]:
function transf_lin_mod_LONG_DUR(data,test_year, lambda, seuil, toPredict, varNames, criteria)
    
    if lambda != 0
        data[!, :new] = data.Duration .^(lambda)
    else 
        data[!, :new] = log.(data.Duration)
    end
    
    train = filter(row -> year(row.Date) != test_year, data)
    train = filter(row -> row.Duration .>= seuil, train)
    test = filter(row -> year(row.Date) == test_year, data)
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
    
    
    l = 1
    for i in combinations
         model = getLinearModel(train, "new", i)
            
        if lambda != 0
            MSE_test = sum( ((complex.(predict(model, test)) ).^(1/lambda) - test.Duration).^2 ) / (nrow(test) )
            #MSE_train = sum( (predict(model, train) .^(1/lambda) - train.new .^(1/lambda)).^2 ) / (nrow(train) -length(β̂))
            #MSE_full = sum( (predict(model, data) .^(1/lambda) - data.new .^(1/lambda)).^2 ) / (nrow(data) -length(β̂))
        else
            MSE_test = sum( (exp.(predict(model, test)) - test.Duration).^2 ) / (nrow(test) )
            #MSE_train = sum( (exp.(predict(model, train)) - exp.(train.new)).^2 ) / (nrow(train) -length(β̂))
            #MSE_full = sum( (exp.(predict(mdoel, data)) - exp.(data.new)).^2 ) / (nrow(data) -length(β̂))
        end
        mse = MSE_test
        
        scores[l] = real(mse)
        println(i, mse)
        l = l+1
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
    
end

In [ ]:
function transf_gamma_mod(data,test_year, lambda, toPredict, varNames, criteria)
    
    train = filter(row -> year(row.Date) != test_year, data)
    test = filter(row -> year(row.Date) == test_year, data)
    
    if lambda != 0
        train[!, :new] = train.Duration .^(lambda)
    else 
        train[!, :new] = log.(train.Duration)
    end
    
    
    
    combinations = Combinatorics.combinations(varNames)
    scores = Vector{Float64}(undef, 2^length(varNames)-1)
    
    
    l = 1
    for i in combinations
        local model
        
        try 
            model = getGammaModel(train, "new", i, "log")
        catch e
            MSE_test = 10000000000
        else
            if lambda != 0
                MSE_test = sum( (predict(model, test) .^(1/lambda) - test.Duration).^2 ) / (nrow(test))
            else
                MSE_test = sum( (exp.(predict(model, test)) - test.Duration).^2 ) / (nrow(test))
            end
        end
            
        
        mse = MSE_test
        
        scores[l] = mse
        println(i, mse)
        l = l+1
    end
    
    
    if criteria == "MSE"
        return sort(DataFrame(Variables = collect(combinations), MSE = scores), :MSE)
        
    elseif criteria == "RMSD"
        return sort(DataFrame(Variables = collect(combinations), RMSD = sqrt.(scores)), :RMSD)
    end
    
end

In [6]:
function cumulative_variance_plot(sigma::Vector{Float64}, plot::Bool)
    cumvar = cumsum(sigma.^2)

    ratio = cumvar / cumvar[end]

    var = DataFrame(k = Int64[], Variance = Float64[])

    for k in 1:length(ratio)
        push!(var, [k, ratio[k]])
    end
    
    if plot
        return plot(var, x=:k, y=:Variance, Geom.line)
    else
        return var
    end

end


function PCA_principal_components(SVD, k)
    
    return SVD.U[:,1:k]*Diagonal(SVD.S[1:k])
    
end

PCA_principal_components (generic function with 1 method)